# Miscellaneous GSL utilities

Some isolated utilities, which sometimes prefigure features to come in the next versions of the C ++ standard and its library.

## Contracts

GSL offers two dedicated assertions to validate *pre-conditions* and *post-conditions*.
* `Expects(p)`: stop the application unless `p == true`.
* `Ensures(p)`: stop the application unless `p == true`.

These assertions are currently implemented via macros, and must be placed in the body of functions, while waiting for future decisions of the standardization committee that deals with contracts and syntax of assertions. In the [Contract Proposal](http://www.open-std.org/jtc1/sc22/wg21/docs/papers/2016/p0380r1.pdf), it is proposed to to move these declarations to the level of function declarations and use some specifiers such as `[[expects: p]]`. The latest news is that the contracts will not be in the C++20.

In [1]:
%%file tmp.gsl-utilities.cpp

#include <iostream>
#include <gsl/gsl>

void divide( double num, double den ) {
    Expects(den!=0) ;
    std::cout<<(num/den)<<std::endl ;
}

int main() {
    divide(5,0) ;
}

Writing tmp.gsl-utilities.cpp


In [2]:
!rm -f tmp.gsl-utilities.exe && g++ -std=c++17 -I./ tmp.gsl-utilities.cpp -o tmp.gsl-utilities.exe

In [3]:
!./tmp.gsl-utilities.exe

terminate called without an active exception
Aborted


## Final action: `finally()`

As a last resort, when the resources management tools are not sufficient, we can define a function to be invoked at the end of a block.

In [32]:
%%file tmp.gsl-utilities.cpp

#include <iostream>
#include <gsl/gsl>

struct Demo {
    Demo() { std::cout<<"Constructor"<<std::endl ; }
    ~Demo() { std::cout<<"Destructor"<<std::endl ; }
} ;

int main() {
    Demo * d = new Demo ;
    auto _ = gsl::finally([d](){ delete d ; }) ;
    // ...
}

Writing tmp.gsl-utilities.cpp


In [26]:
!rm -f tmp.gsl-utilities.exe && g++ -std=c++17 -I./ tmp.gsl-utilities.cpp -o tmp.gsl-utilities.exe

In [27]:
!./tmp.gsl-utilities.exe

Constructor
Destructor


## "C-style" strings `gsl::zstring` and `gsl::czstring`

The two aliases below simply allow to draw attention to strings that are supposed to end with a null character:
* `zstring`: `char *`, being `nullptr` or pointing to a C-style string;
* `czstring`: `char const *`, being `nullptr` or pointing to a C-style string.

If your C-style string pointer should not be null, use `gsl::not_null<zstring>`.

If your string is not supposed to end with a null character, use `string_view`.

## Numerical utilities: `narrow_cast<T>(x)` and `narrow<T>(x)`

The first one, `narrow_cast<T>(x)`, is just a statement for programmers or testing tools. It clarifies the fact that the developer wants **voluntarily** to force a value to a less precise type. 

The second one, `narrow<T>(x)`, checks at runtime that the value `x` was not narrowed when transformed into a `T`, and throws an exception (or terminates the program) otherwise (if `static_cast<T>(x) != x`). Shouldn't it be called instead `no_narrow` ?!

In [1]:
%%file tmp.gsl-utilities.cpp

#include <iostream>
#include <gsl/gsl>

int main() {
    double d = 3.14 ;
    int i = gsl::narrow<int>(d) ;
    std::cout<<i<<std::endl ;
}

Writing tmp.gsl-utilities.cpp


In [2]:
!rm -f tmp.gsl-utilities.exe && g++ -std=c++17 -I./ tmp.gsl-utilities.cpp -o tmp.gsl-utilities.exe

In [3]:
!./tmp.gsl-utilities.exe

terminate called without an active exception
Aborted (core dumped)


# Questions ?

## Exercise 1

Create your own `no_narrow`. Some hints:
* in order to verify that there is no "narrowing", it is necessary to verify that `static_cast<T>(x) == x` ;
* as you want to be able to build a `no_narrow` from a value of any other type, you have to template the constructor...

In [31]:
%%file tmp.gsl-utilities.cpp

#include <iostream>
#include <cassert>

// ... no_narrow ...
 
int main()
 {     
  double d1 = 42 ;
  int i1 = no_narrow<int>(d1) ;
  std::cout<<i1<<std::endl ;
    
  double d2 = 3.14 ;
  int i2 = no_narrow<int>(d2) ;
  std::cout<<i2<<std::endl ;
 }

Overwriting tmp.gsl-utilities.cpp


In [ ]:
!rm -f tmp.gsl-utilities.exe && g++ -std=c++17 -I./ tmp.gsl-utilities.cpp -o tmp.gsl-utilities.exe

In [ ]:
!./tmp.gsl-utilities.exe

## Exercise 2

Create your own `my_finally`. We suggest to use two following elements:
* On the one hand, a `FinalAction` structure is capable of storing a function and a callable which invokes this function when it is destroyed.
* On the other hand, a utility function `my_finally` is used to create an object of this type.

Why not to settle for only one structure?

In [ ]:
%%file tmp.gsl-utilities.cpp

#include <iostream>
 
// ... struct ActionFinal ...
 
// ... function my_finally ...

int main()
 {     
  std::cout<<"Step 1"<<std::endl ;
  auto _ = my_finally([](){ std::cout<<"Step 3"<<std::endl ; }) ;
  std::cout<<"Step 2"<<std::endl ;
 }

In [ ]:
!rm -f tmp.gsl-utilities.exe && g++ -std=c++17 -I./ tmp.gsl-utilities.cpp -o tmp.gsl-utilities.exe

In [ ]:
!./tmp.gsl-utilities.exe

## Sources

* https://stackoverflow.com/questions/40127965/how-exactly-is-stdstring-view-faster-than-const-stdstring
* http://isocpp.github.io/CppCoreGuidelines/CppCoreGuidelines#gsl-guidelines-support-library
* http://modernescpp.com/index.php/c-core-guideline-the-guidelines-support-library
* http://nullptr.nl/2018/08/refurbish-legacy-code/

© *CNRS 2021*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*